# Lightcurves of Transiting Exoplanets

#### By [Dr. Daniel Apai](http://apai.space) and [Dr. Ben Rackham](http://rackham.space)

# 1. Introduction

In  this  activity  we  will  explore  how  some  extrasolar  planets  can  be  discovered  and characterized by precisely measuring the brightness of their host stars. When planets pass in front of their host stars (as seen from Earth) they cover part of the host star’s disk — making it look  a  tiny  bit  fainter.  These  events  are  called **planetary  transits**.  The  slight  dimming  has  a characteristic shape and Project EDEN searches for such events to find the planets that cause them.  NASA’s  Kepler  Space  Observatory  has  used  exoplanet  transit  events  to  identify  over 4,000 new extrasolar planets – the largest bounty of new worlds ever!

We will explore how transits can be analyzed — not only to identify the presence of a planet, but  also  to  learn  about  its  basic  properties:  size,  orbital  period,  and  —  with  some  additional information — density and likely composition.

## First let’s clarify a few key concepts:

**Lightcurve:** a figure that shows the brightness of an object (here the host star) over time. For a typical star, this would be a flat line: the star is always the same brightness.

**Transit Event:** an event during which a planet passes in front of its host star (as seen from Earth), blocking a fraction of light from star and causing it to appear dimmer. 

**Transit Depth:** the maximum brightness difference between the in-transit and out-of-transit brightness of the star.

**Ingress:** The first stage of the transit, during which only part of the planet is only obscuring the star.

**Egress:** The final stage of the transit, during which only part of the planet is only obscuring the star.

# 2. Interpreting a Lightcurve with a Transit

In this activity, we will use the Python package `batman` to simulate transit lightcurves.

Let's start by importing some necessary Python packages.

To do so, highlight the cell below and press `Shift+Enter` to run the code. This is also how we will run all following snippets of code throughout this exercise.

In each of the code cells below, anything following the the `#` sign is a comment. It is meant to help you read the code, but it is not read by the computer.

In [ ]:
# Import some packages for displaying results
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interactive

# Import standard Python science packages
import astropy.units as u
import numpy as np

# Import the transit modeling package
import batman

Run the cell below to adjust the appearance of the figures we will plot later in this exercise.

In [ ]:
# Set the appearance of the figures
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 18

To better understand how to use the `batman` package to model transit lightcurves, let's make an example lightcurve of a **hot Jupiter**: a Jupiter-sized exoplanet on a very short orbit.

First, we will initialize a `params` object to hold the orbital parameters of our example planet. The parameters provided below describe the orbit of the planet and its orientation, which affect the exact shape of the transit and how symmetric it is.

In [ ]:
params = batman.TransitParams()                  # Creates a new params object
params.t0 = 0.                                   # time of inferior conjunction
params.per = 3.                                  # orbital period (days)
params.rp = ((1.0*u.Rjup)/(1.0*u.Rsun)).si.value # planet radius (in units of stellar radii)
params.a = 15.                                   # semi-major axis (in units of stellar radii)
params.inc = 87.                                 # orbital inclination (in degrees)
params.ecc = 0.                                  # eccentricity
params.w = 90.                                   # longitude of periastron (in degrees)
params.u = [0.1, 0.1]                            # limb darkening coefficients [u1, u2]
params.limb_dark = "quadratic"                   # limb darkening model

Now let's define the time points we're interested in and generate a model of the transit event.

In [ ]:
time_1 = np.linspace(-0.025, 0.025, 1000)*params.per # Creates an array of 1000 points 
                                                     # near the transit time
model_1 = batman.TransitModel(params, time_1)        # Creates the model

And finally, we will calculate the light curve.

In [ ]:
flux_1 = model_1.light_curve(params) # Calculates the light curve

Now let's look at the results.

In [ ]:
plt.figure()                                            # Makes a new figure
plt.plot(time_1*24, flux_1)
plt.title("Figure 1: Example Transit of a Hot Jupiter")
plt.xlabel("Hours from mid-transit")
plt.ylabel("Star brightness (normalized)")
plt.show()

##### Question 1: Check Figure 1 above. What is your estimate the transit depth? Approximately how long are the ingress/egress stages?

# 3. Determining the Orbital Period

The **orbital period** is simply the time that it takes for a planet to complete an orbit. For Earth this is one year, but some planets have much shorter and some planets have much longer orbital periods. Once the orbital period of a planet and the mass of its host star are known, it is possible to calculate the radius of its orbit—in other words, how far is it from its host star.

##### Question 2: How do you think the orbital period can be measured for a transiting planet from its host star’s lightcurve?

Now let's look at an example of a lightcurve with multiple transits of the same planet.  We'll use the same hot Jupiter parameters as before, but we'll define a new time array, `time_2`, which covers a longer time period so that we can visualize multiple transits.

In [ ]:
time_2 = np.linspace(-2.5, 2.5, 1000)*params.per
model_2 = batman.TransitModel(params, time_2)
flux_2 = model_2.light_curve(params)

Now we'll plot it.

In [ ]:
plt.figure()
plt.plot(time_2, flux_2)
plt.title("Figure 2: Multiple Transits of a Hot Jupiter")
plt.xlabel("Days from mid-transit")
plt.ylabel("Star brightness (normalized)")
plt.show()

Now let's take a look at what transit lightcurves look like for planets with different orbital periods.

Run the code in the cell below to generate an interactive version of the transit model.

In [ ]:
def transit_model_1(period):
    """
    Function for interactive transit modeling
    """
    # Parameters of planet model
    params = batman.TransitParams()
    params.t0 = 0.                                   # time of inferior conjunction
    params.per = period                                  # orbital period (days)
    params.rp = ((1.0*u.Rjup)/(1.203*u.Rsun)).si.value # planet radius (in units of stellar radii)
    params.a = 15.                              # semi-major axis (in units of stellar radii)
    params.inc = 87.                                 # orbital inclination (in degrees)
    params.ecc = 0.                                  # eccentricity
    params.w = 90.                                   # longitude of periastron (in degrees)
    params.u = [0.1, 0.1]                            # limb darkening coefficients [u1, u2]
    params.limb_dark = "quadratic"                   # limb darkening model

    # Defining the timeframe and calculating flux
    time = np.linspace(-1, 1, 10000)*6        # Creates an array of 10000 points 
                                              # over a roughly 12-day period
    model = batman.TransitModel(params, time)
    flux_model = model.light_curve(params)

    # Plotting results
    plt.figure()
    plt.plot(time, flux_model, 'r-')
    plt.title("Figure 3: Multiple Transits of a Hot Jupiter with a {:0.1f}-day Period".format(params.per))
    plt.xlabel("Days from mid-transit")
    plt.ylabel("Star brightness (normalized)")
    plt.show()

print('Using the slider below, vary the orbital period (in units of days):')
interactive_plot = interactive(transit_model_1, period=(0.5, 5, 0.1))
display(interactive_plot)

##### Question 3: Is it easier to find transiting exoplanets with shorter or longer orbital periods?  Why?

# 4. Determining the Size of a Planet 

The larger the planet the larger the fraction of its host star that it can cover. A planet as large as its host star (an unrealistic example) could cover the entire star from view. In reality, most planets are very smaller, compared  to even the smallest stars. Therefore, they only cover part a small fraction of the host star’s disk.

Run the code below to draw a view of a planetary transit as seen from Earth.

In [ ]:
# Planet and star parameters
m_planet = 0.01
m_star = 1.
r_planet = 0.2
r_star = 1.

# Plotting values
fig, ax = plt.subplots(figsize=(8, 8))
star = plt.Circle((0,0), r_star)
ax.add_artist(star)
ax.plot([0, r_star], [0, 0], 'k-')
ax.text(r_star/2., 0, '$R_{s}$', va='top', ha='center')
planet = plt.Circle((-0.35, 0.35), r_planet, color='white')
ax.add_artist(planet)
ax.plot([-0.35, -0.35+r_planet], [0.35, 0.35], 'k-')
ax.text(-0.35+r_planet/2., 0.35, '$R_{p}$', va='top', ha='center')
ax.set_xlim(-r_star, r_star)
ax.set_ylim(-r_star, r_star)
plt.axis('off')
plt.show()

The circles represent the star and the planet. The radius of the star ($R_{*}$) and the radius of the planet ($R_{p}$) are shown on the figure. The mass of the star is $M_{*}$ and the mass of the planet is $M_{p}$.

*Modify the code above and re-run the cell to plot results for different values of `m_planet`, `m_star`, `r_planet`, and `r_star`.*

*Discuss with your partner which factors influence the transit depth.*

#### Question 4: Which of the following formulae may describe best the transit depth and why?

**A)**  $D = (M_{*}^{2} R_{*}) / (M_{p}^{2} R_{p})$ 

**B)**  $D = R_{*} / R_{p}$

**C)**  $D = M_{*} / M_{p}$ 

**D)**  $D = R_{p}^{2} / R_{*}^{2}$ 

# 5. The Transit of HD 209458 b

Now let's figure out the size of an actual transiting exoplanet!

Run the code in the cell below to plot an observed transit lightcurve of the exoplanet HD 209458 b.

In [ ]:
# Loading observed data
time_obs, flux_obs = np.loadtxt('HD209458b.dat', unpack=True)

# Plotting results
plt.figure()
plt.errorbar(time_obs, flux_obs, 0.001, marker='o', ls='', color='b', label='Observed')
plt.title("Figure 4: Observations of HD 209458 b")
plt.legend(loc=4)
plt.xlabel("Days from mid-transit")
plt.ylabel("Star brightness (normalized)")
plt.show()

The lightcurve shows the exoplanet transit, but it also displays scatter due to noise sources that are inherent in any observation.  We will fit different models against these noisy data to make our best estimate of the planet's radius.

Run the code in the cell below to generate a new interactive version of the transit model.

In [ ]:
def transit_model_2(radius):
    """
    Function for interactive transit modeling
    """
    # Parameters of planet model
    rp = ((radius*u.Rjup)/(1.203*u.Rsun)).si.value
    params = batman.TransitParams()
    params.t0 = 0.                                 # time of inferior conjunction
    params.per = 3.                                # orbital period (days)
    params.rp =  rp                                # planet radius (in units of stellar radii)
    params.a = 15.                                 # semi-major axis (in units of stellar radii)
    params.inc = 87.                               # orbital inclination (in degrees)
    params.ecc = 0.                                # eccentricity
    params.w = 90.                                 # longitude of periastron (in degrees)
    params.u = [0.1, 0.1]                          # limb darkening coefficients [u1, u2]
    params.limb_dark = "quadratic"                 # limb darkening model

    # Defining the timeframe and calculating flux
    time = np.linspace(-0.025, 0.025, 100)*params.per # Creates an array of 100 points 
                                                      # near the transit time
    model = batman.TransitModel(params, time)            # Initializes model
    flux_model = model.light_curve(params)            # Calculates light curve

    # Loading observed data
    time_obs, flux_obs = np.loadtxt('HD209458b.dat', unpack=True)

    # Plotting results
    plt.figure()
    plt.errorbar(time_obs, flux_obs, 0.001, marker='o', ls='', color='b', label='Observed')
    plt.plot(time, flux_model, 'r-', label='Model')
    plt.title("Figure 5: HD 209458 b")
    plt.legend(loc=4)
    plt.xlabel("Days from mid-transit")
    plt.ylabel("Star brightness (normalized)")
    plt.show()

print('Using the slider below, vary the exoplanet radius (in units of Jupiter radii)' + 
      ' to find the best model to match the data:')
interactive_plot = interactive(transit_model_2, radius=(0.1, 2, 0.1))
display(interactive_plot)

##### Question 5: What is the best-fit radius for HD 209458 b? 

# 6. Detecting Transits of Earth-sized Exoplanets

In this final exercise, we will explore how we can detect transits of exoplanets the size of Earth.

Run the code below to generate an interactive verision of the transit model that will let you vary the both radius of the planet ($R_{p}$) and the radius of the star ($R_{s}$). Each time you change a parameter, it will show you the true transit model and make a new simulated observational dataset.

In [ ]:
def transit_model_3(Rp, Rs):
    """
    Function for interactive transit modeling
    """
    # Parameters of planet model
    params = batman.TransitParams()
    params.t0 = 0.                                   # time of inferior conjunction
    params.per = 3.                                  # orbital period (days)
    params.rp = ((Rp*u.earthRad)/(Rs*u.Rsun)).si.value # planet radius (in units of stellar radii)
    params.a = 15.                              # semi-major axis (in units of stellar radii)
    params.inc = 87.                                 # orbital inclination (in degrees)
    params.ecc = 0.                                  # eccentricity
    params.w = 90.                                   # longitude of periastron (in degrees)
    params.u = [0.1, 0.1]                            # limb darkening coefficients [u1, u2]
    params.limb_dark = "quadratic"                   # limb darkening model

    # Defining the timeframe and calculating flux
    time = np.linspace(-0.025, 0.025, 100)*params.per # Creates an array of 100 points 
                                                      # near the transit time
    model = batman.TransitModel(params, time)         # Initializes model
    flux_model = model.light_curve(params)            # Calculates light curve
    noise = np.random.normal(0, 0.001, flux_model.shape)
    flux_obs = flux_model + noise

    # Plotting results
    plt.figure()
    plt.errorbar(time, flux_obs, 0.001, marker='o', ls='', color='b', label='Observed')
    plt.plot(time, flux_model, 'r-', label='Model')
    plt.title("Figure 6: Transit of a {:0.1f}-Earth-radius planet before a {:0.1f}-Sun-radius star".format(Rp, Rs))
    plt.legend(loc=4)
    plt.xlabel("Days from mid-transit")
    plt.ylabel("Star brightness (normalized)")
    plt.show()

print('Using the slider below, vary the exoplanet radius (in units of Earth radii)' + 
      ' and the star radius (in units of solar radii):')
interactive_plot = interactive(transit_model_3, Rp=(0.5, 10, 0.5), Rs=(0.1, 2, 0.1))
display(interactive_plot)

##### Question 6:  With this level of noise, how small does the star have to be to reliably detect the transit of an Earth-sized ($R_{p} = 1.0 R_{Earth}$) exoplanet?